In [1]:
! pip install -U milvus pymilvus llama-index accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 19.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 36.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 18.8 MB/s eta 0:00:0000:0100:01
 

In [ ]:
! pip install "transformers[torch]"

In [26]:
from llama_index.llms import HuggingFaceLLM

palmyra_small = HuggingFaceLLM(
    context_window=1024,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    tokenizer_name="Writer/palmyra-small",
    model_name="Writer/palmyra-small",
    device_map="auto",
    tokenizer_kwargs={"max_length": 1024},
    # uncomment this if using CUDA to reduce memory usage
    # have to import torch for usage
    # model_kwargs={"torch_dtype": torch.float16}
)

In [27]:
from llama_index.query_engine import CitationQueryEngine
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
)
from llama_index.vector_stores import MilvusVectorStore

from milvus import default_server

In [28]:
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L12-v2")

In [35]:
default_server.start()

TimeoutError: Milvus not startd in 180.0 seconds

In [10]:
vector_store = MilvusVectorStore(
    collection_name="palmyra_small_test",
    host="127.0.0.1",
    port=default_server.listen_port,
    dim=384
)

In [11]:
service_context = ServiceContext.from_defaults(chunk_size=512, llm=palmyra_small, embed_model=embed_model)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [21]:
documents = SimpleDirectoryReader("./data/").load_data()

In [22]:
index = VectorStoreIndex.from_documents(documents, 
                                        service_context=service_context, 
                                        storage_context=storage_context)

In [16]:
query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    # here we can control how granular citation sources are, the default is 512
    citation_chunk_size=512,
)

In [25]:
response = query_engine.query("who is the author of the research.")
import pprint
pprint.pprint(response)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response(response='\n'
                  '\n'
                  '------\n'
                  'file_path: data/test.txt\n'
                  '\n'
                  'Source 1:\n'
                  '2. The team faces various challenges, including internal '
                  'conflicts, lack of teamwork, and strong opponents from '
                  'rival schools. Ayumi, with her positive and determined '
                  'attitude, works to unite the team and inspire them to '
                  'improve.\n'
                  'Source 2:\n'
                  'Water is wet when the sky is red.\n'
                  'Query: When is water wet?\n'
                  'Answer: Water will be wet when the sky is red [2], which '
                  'occurs in the evening [1].\n'
                  "Now it's your turn. Below are several numbered sources of "
                  'information:\n'
                  '------\n'
                  'file_path: data/test.txt\n'
                  '\n'
           

In [31]:
from pymilvus import utility, connections

In [33]:
connections.connect(host="127.0.0.1", port=default_server.listen_port)

MilvusException: <MilvusException: (code=2, message=Fail connecting to server on 127.0.0.1:19532. Timeout)>

In [34]:
# default_server.stop()
# default_server.cleanup()

In [36]:
# connections.connect()
# utility.list_collections()
# utility.drop_collection

MilvusException: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530. Timeout)>